In [3]:
import pandas as pd

df = pd.read_csv("../sampel/sampel_bersih.csv")
df.head()

,content,label
0,game grafik pukau gameplay nya seru event naru...,positive
1,skin gratis event bikin semangat main terima k...,positive
2,main mlbb bareng temen seru fitur tim bikin ko...,positive
3,event collab nya keren abis naruto mini game n...,positive
4,grafik update baru halus efek skill hero spekt...,positive


### Tes Bagi manual data train dan test

In [4]:
# Ambil data training (80 per label)
df_train_neg = df[df['label'] == 'negative'].sample(n=80, random_state=1)
df_train_pos = df[df['label'] == 'positive'].sample(n=80, random_state=1)
df_train_net = df[df['label'] == 'neutral'].sample(n=80, random_state=1)

df_train = pd.concat([df_train_neg, df_train_pos, df_train_net]).sample(frac=1, random_state=1).reset_index(drop=True)

# Ambil data testing (20 per label)
df_test_neg = df[df['label'] == 'negative'].drop(df_train_neg.index).sample(n=20, random_state=2)
df_test_pos = df[df['label'] == 'positive'].drop(df_train_pos.index).sample(n=20, random_state=2)
df_test_net = df[df['label'] == 'neutral'].drop(df_train_net.index).sample(n=20, random_state=2)

df_test = pd.concat([df_test_neg, df_test_pos, df_test_net]).sample(frac=1, random_state=2).reset_index(drop=True)

### Cek Hasil

In [6]:
print(df_train['label'].value_counts())
print(df_test['label'].value_counts())

label
neutral     80
positive    80
negative    80
Name: count, dtype: int64
label
positive    20
negative    20
neutral     20
Name: count, dtype: int64


### TF-IDF Vectorizer

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF hanya dari data training
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(df_train['content'])
X_test = vectorizer.transform(df_test['content'])


### ✅ Ubah Label ke One-Hot

In [8]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

le = LabelEncoder()
y_train_enc = le.fit_transform(df_train['label'])
y_test_enc = le.transform(df_test['label'])

y_train_cat = to_categorical(y_train_enc)
y_test_cat = to_categorical(y_test_enc)

### ✅ Buat & Latih Model DNN

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')  # 3 kelas: negative, neutral, positive
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.fit(X_train.toarray(), y_train_cat, epochs=10, batch_size=16, validation_split=0.2)

Epoch 1/10


C:\Users\agnes\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.4398 - loss: 1.0899 - val_accuracy: 0.5625 - val_loss: 1.0634
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8251 - loss: 1.0080 - val_accuracy: 0.6458 - val_loss: 1.0101
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8975 - loss: 0.8925 - val_accuracy: 0.7708 - val_loss: 0.8989
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9570 - loss: 0.7013 - val_accuracy: 0.8125 - val_loss: 0.7149
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9823 - loss: 0.4559 - val_accuracy: 0.9375 - val_loss: 0.5066
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 0.2721 - val_accuracy: 0.9583 - val_loss: 0.3187
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 1.0000 - loss: 0.1272 - val_accuracy: 0.9792 - val_loss: 0.2055
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 1.0000 - loss: 0.0550 - val_accuracy: 0.9792 - val_loss: 0.

In [10]:
loss, acc = model.evaluate(X_test.toarray(), y_test_cat)
print(f"🎯 Akurasi: {acc:.2%}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 1.0000 - loss: 0.0421 
🎯 Akurasi: 100.00%


In [12]:
import joblib

joblib.dump(vectorizer, 'tfidf_vectorizer_DNN.pkl') 
model.save('sentiment_DNN_v2.keras')

In [27]:
ulasan_baru = ["anak dajal"]

# Preprocessing: transform ulasan ke tf-idf
X_new = vectorizer.transform(ulasan_baru)

# Prediksi probabilitas dan kelas
y_pred = model.predict(X_new.toarray())
pred_label = le.inverse_transform([y_pred.argmax()])

print("Prediksi sentimen:", pred_label[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Prediksi sentimen: positive


In [ ]:
import matplotlib.pyplot as plt

history = model.fit(...)  # Simpan hasil training ke variabel history

# Plot akurasi
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Val')
plt.title('Akurasi')
plt.xlabel('Epoch')
plt.ylabel('Akurasi')
plt.legend()
plt.show()

# Plot loss
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Val')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()